## HTTP 爬虫笔记

> $Requests:\ HTTP\ for\ Humans^{TM}$

- Requests, 强大、简洁而优雅
- 作者是**Kenneth Reitz**大牛，其才华和形象都令人印象深刻
- 个人一般用来做区别于浏览器模拟的 HTTP 请求爬虫，速度当然就要比 selenium 快多了
- 如果想更快就要用到 aiohttp, gevent 等异步 requests 版本了

In [ ]:
import requests

### 伪装请求头

首先要尽量伪装请求头（Request Headers），避免被 ban

**这里还可以使用 faker 库生成指定类型的随机 User-Agent**

In [ ]:
# 常用伪装请求头，根据人工正规登录后的信息修改后再使用。
headers = {
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding':'utf-8',
    'Accept-Language':'zh-CN,zh;q=0.9,en;q=0.8',
    'Cache-Control':'max-age=0',
    'Connection':'keep-alive',
#    'Cookie':'',
    'Host':'',
    'Referer':'',
    'Upgrade-Insecure-Requests':'1',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3573.0 Safari/537.36',
}

cookies = {'cookie_name':'cookie_value'}  # cookie可以写在 headers 里面，也可以单独另外给出

### 发起请求

使用伪装请求头和登录 cookies 打开目标网站，绕过登录步骤。缺点是 cookie 需要每次手工登录获取，而且有效期不一定。

还可以通过 params 参数传入一个 dict，用于拼接 URL 的查询参数

In [ ]:
payload = {'a':'1'}
r = requests.get('target_url', cookies=cookies, headers=headers, params = payload )
print(r.url) # 结果为 target_url?a=1

### 页面分析

当然可以使用`beautiful soap`, `etree`等包对 html 树结构进行分析。如果目标位置上下文的文本唯一显著，那也可以直接用如下方法

假设想要获取的信息在文本"anchor"后的10位数字

In [ ]:
anc = r.text.index('anchor') # 将索引原点钉在某个方便唯一定位的锚文本上
target = r.text[anc+6:anc+16] # anchor第一个字母a位置为0

### 下载文件

最直接的可以使用 Python 自带的 open 方法来读写文件

In [ ]:
with open(file,'w',encoding='utf-8') as f: # 第二个参数换成 wb 即可二进制写入，但就不能指定编码了
    f.write(yourcontent) # with open 结尾不需要 f.close()


优点是可以控制写入文件的全过程（特别是编码），必须指定文件名和文件类型。

但有时候并不想这么做，只是想直接把下载链接给出的文件原样下载下来。那么就可以使用 urlretrieve

In [ ]:
import urllib.request # 另一个request, urlretrieve位于此模块内

urllib.request.urlretrieve(download_url, file_path)